<a href="https://colab.research.google.com/github/HomayounfarM/Classification/blob/main/Decision_tree/Decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision tree

## Decision tree
Decision tree is a **non-parametric**, **supervised**, **classification** algorithm that assigns data to discrete groups.

**Non-parametric**: Decision tree does NOT make assumptions about data’s distribution or structure.

**Supervised**: The class of training set MUST be provided by the users.

**Classification**: Decision tree classifies data into discrete classes, instead of predicting a real number.

**Note**:The workflow of regression tree is similar to the decision tree, but regression tree predicts continuous values.

**Root node** and **inner nodes** represent a test/question on an attribute, while root node is the beginning of the tree (first question to be asked). Further, a **branch** represents an outcome of a test, which is used to connect the nodes. Last, the **leaf nodes** are the classification result. There is no more question in the leaf nodes, but all the data in a leaf node is classified to the same class.

### Parameters (**criterion**, **max_depth**, **class_weight**)

Decision tree has a lot of parameters we can tune, but we’ll just go through the frequently used ones, **criterion**, **max_depth**, and **class_weight**.

**criterion**: As we talked about, root node and inner nodes represent a test on an attribute. However, how do we decide which attribute to split on first (i.e. age, gender)? We can use **Gini impurity** or **Information gain** to determine . 

**max_depth**: **First question**, what is tree depth? Tree depth is the number of layers of a tree.

**Second question**, why would we want to limit the depth of tree? We want to avoid overfitting.

**Note**: Limiting tree depth is a method of pre-pruning. Namely, we prune the tree before it even grows to avoid the issue of overfitting!
In fact, there are other pre-pruning measures, such as min_samples_split (minimum number of samples required to split an internal node) and min_samples_leaf (minimum number of samples required to be at a leaf node). There is no standard process in terms of selecting pre-pruning measures, we have to trial-and-error to determine which parameters to use.

**class_weight**: the weights of the classes will affect the splitting criterion. To put it simply, splitting criterion will pay more attention to the “important” class and try to classify all the data in that class correctly. Thus, we can assign more weight to the conceptually important classes, which might differ according to the purpose of the research.

**Note**: One way to choose hyper-parameters is using grid search and K-fold cross validation

**Problem Statement:** we want to predict whether a passenger will survive in the Titanic (y) given the characteristics of the passenger (x). A decision tree might look like this.





In [ ]:
import requests
from io import BytesIO
url = 'https://github.com/HomayounfarM/Classification/blob/main/Decision_tree/photo/tree.jpg?raw=true'
page = requests.get(url)
Image.open(BytesIO(page.content))

In [ ]:
# Import libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder          
from sklearn.model_selection import train_test_split  
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import matplotlib.pyplot as plt

In [ ]:
# Importing data
url = 'https://raw.githubusercontent.com/HomayounfarM/Classification/main/Decision_tree/Data/titanic.txt'
data = pd.read_csv(url)
data.info()

**Note:** axis=1 means we’re dropping a column, inplace=True means we want to make modification on the original dataset.

In [ ]:
# Manage null data, dropping the whole column, 'Cabln'
print(data.isnull().sum())
# drop embarked
data.drop('Cabin', axis=1, inplace = True)
data.isnull().sum()

In [ ]:
# Count the number of na
data.isna().sum()

In [ ]:
# drop na 
data.dropna(inplace=True)
data.isna().sum()

In [ ]:
data.info()

Now, let's look at the first few rows of the data

In [ ]:
data.head()

We find the values in Sex are **strings**, ‘female’ and ‘male’. Decision tree only takes columns of numerical values, whether continuous or not. Thus, we have to convert the string to numbers, and we use **LabelEncoder** to do the conversion.

In [ ]:
le = LabelEncoder()
data['Sex']=le.fit_transform(data['Sex'])
data.head()

Decide independent variables and target variables

In [ ]:
X = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]
y = data['Survived']

Train/Test split:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=65)

Choose the hyper-parameters and train a model
we just manually select the hyper-parameters. In specific, we choose maximum tree depth as 3. In other words, we leave all the other parameters as default, and you can press tab+shift to see the default values of DecisionTreeClassifier class.

In [ ]:
dt = DecisionTreeClassifier(max_depth=3)
dt_model = dt.fit(x_train, y_train)

Visualize the tree:

To have a more concrete idea, let’s visualize it! In Python, we can simply use plot_tree to visualize the tree model. To illustrate, feature_names is the names of independent variables, and class_names is the values/classes in the target attribute.

In [ ]:
fig = plt.figure(figsize=(12, 10))
tree.plot_tree(dt_model, feature_names=list(X.columns), class_names=['Not survived', 'Survived'])
plt.show()

Here’s our tree! Indeed it only has 3 layers since we set max_depth as 3. Further, the first question our tree asks is “Is the sex of passenger ≤ 0.5?”. It seems like a weird question, but we did encode the gender attribute in step 3. The female is represented by 0; the male is represented by 1. Thus, this question should be interpreted as “Is the passenger female?”.

Evaluate the model performance on testing set:

The accuracy score on testing set is 0.783. Namely, 78.3% of testing set is correctly classified, which shows the model is generally fine. However, it’s always important to examine model’s performance on individual class. Therefore, we should also check out precision and recall score (see this article for detail).

In [ ]:
print("Accuracy score on Testing set: ", dt_model.score(x_test, y_test))

## In the following cells we discuss some ways manage NaN and encoding categorical veriables.


In [ ]:
# Example
# Managing NaN by removing the whole row
df = pd.DataFrame({'class':[np.nan,'m', 'l', 'h', 'h', 'm', 'l', 'h', 'm', 'l'], 'value':[5,1, 2, 3, 4, 5, 7, 8, 9, 10]})
print(df)
df.dropna(inplace=True)
df

In [ ]:
# Example
# Note: When we have a categorical variable with more then two categories, 
# we can use OneHotEncoder() instead of LabelEncoder(). See the following example.
df = pd.DataFrame({'class':['m', 'l', 'h', 'h', 'm', 'l', 'h', 'm', 'l']})
le = LabelEncoder()
df['class'] = le.fit_transform(df['class'])
df

In [ ]:
# Example
# The extra bracket in df[['class']] reshape it from (n,) to (n,m)
df = pd.DataFrame({'class':['m', 'l', 'h', 'h', 'm', 'l', 'h', 'm', 'l']})
onehotencoder = OneHotEncoder()
transformed = onehotencoder.fit_transform(df[['class']]).toarray()
transformed
df[['High', 'Low', 'Medium']] = transformed
df